# Specific Heat Capacity
The specific heat capacity was measured in a temperature sweep from 40 °C to 290 °C. Measured specific heat capacities and temperatures are used for a nonlinear regression in order to fit a polynom of form (Hirono and Hamada, 2010):  

$$c_p(T) = a + bT + cT^2 + dT^{-1} + eT^{-2}$$ 

with $c_p$ in J kg$^{-1}$ K$^{-1}$ and $T$ in K.  

We also use a simpler equation to fit the $c_p$ data (Clauser 2003):  

$$c_p(T) = a + bT + cT^2$$  

with $c_p$ in J kg$^{-1}$ K$^{-1}$ and $T$ in °C. Equations can be chosen with a `boolean` in the function:  

```python
def cp_plot(dat,n,q2):
    if q2==False:
        use Hirono and Hamada, 2010
    if q2==True:
        use Clauser 2003
```

In [43]:
import os, sys
import matplotlib.pyplot as plt
import matplotlib as m
import numpy as np
import itertools as it
%matplotlib inline

from scipy import optimize

In [45]:
# function for cp fit
def fun(x,a,b,c,d,e):
    return a + b*x + c*x**2 + d/x + e/x**2
def fun2(x,a,b,c):
    return a + b*x + c*x**2

def cp_plot(dat,n,q2,title):
    """
    Function to calculate and plot the specific heat capacity dependent on temperature.
    **Arguments:**
        *dat*: array containing temperature and corresponding cp data
        *n*: integer number of points to be ommitted in the plot
        *q2*: boolean. If true, we use just the quadratic fit fun2. 
                       If False, the complete fit fun is used.
    """
    if q2 == False:
        T = dat[:,0]+273.15 # Temperature data in K
        cp = dat[:,1]*1000 # cp data in J/(kg K)
        if title == 'MR03':
            tsim = np.linspace(40,180,500)+273.15 # regression temperature in K
        else:
            tsim = np.linspace(40,300,500)+273.15 # regression temperature in K
    
        popt, pcov = optimize.curve_fit(fun,T,cp) # nonlinear regression
    
        perr = np.sqrt(np.diag(pcov))
        cpsim = fun(tsim,popt[0],popt[1],popt[2],popt[3],popt[4]) # fitted cp
    elif q2 == True:
        T = dat[:,0]
        cp = dat[:,1]*1000
        
        tsim = np.linspace(40,250,500)
        popt, pcov = optimize.curve_fit(fun2,T,cp)
        perr = np.sqrt(np.diag(pcov))
        
        cpsim = fun2(tsim,popt[0],popt[1],popt[2])
    
    #plot
    fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(7,7))
    
    ax.scatter(T[::n],cp[::n],s=60, alpha=0.8,label="measured data") # plot just every nth point
    ax.plot(tsim,cpsim,'r-',linewidth=4, alpha=0.5, label="fit")
    ax.legend(fontsize=16,loc=4)
    if q2 == False:
        ax.set_xlabel("Temperature [K]")
    elif q2 == True:
        ax.set_xlabel("Temperature [$^\circ$C]")
    ax.set_ylabel("specific heat capacities J kg$^{-1}$ K$^{-1}$")
    ax.set_title(title)
    m.rcParams.update({'font.size':15})
    
    if q2 == False:
        ax.text(600, max(cpsim), 'c$_p$(T) = {0:.3f} + {1:.3f} T + {2:.5f} T$^2$+ {3:.3f} T$^-$$^1$ + {4:.3f} T$^-$$^2$'.format(*tuple(popt)))
    elif q2 == True:
        ax.text(350, max(cpsim), 'c$_p$(T) = {0:.3f} + {1:.3f} T + {2:.5f} T$^2$ '.format(*tuple(popt)))
    fig.tight_layout()
    plt.show()

In [46]:
dat = np.genfromtxt('data/03_cp.dat',skip_header=1)

In [48]:
cp_plot(dat,30,False,'MR03')

ValueError: array must not contain infs or NaNs